# Importing Libraries

In [150]:
# Data Visualization
import matplotlib.pyplot as plt

# Data Processing
import pandas as pd

# NLP 
import re

# ML
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

# BERT
from simpletransformers.classification import MultiLabelClassificationModel

import logging
import warnings

# Preprocessing Data for Bidirectional Multilabel Classification using BERT

In [151]:
df = pd.read_pickle("../data/preprocessed_dataset")

In [152]:
df.head()

,questionTitle,questionText,questionLink,topic,answerText,upvotes,views,root_topic,root_multi_label,reflection
0,How can I keep a long distance relationship go...,We weren't long distance until he joined the m...,https://counselchat.com/questions/how-can-i-ke...,relationships,Hello. You are asking a very good question abo...,9,481,family_conflicts,"[1, 0, 1]",You are asking a very good question about how ...
1,How can I ask my boyfriend about who he's text...,We've been in a long distance relationship for...,https://counselchat.com/questions/how-can-i-as...,relationships,I agree with Sherry that in a close intimate r...,9,472,family_conflicts,"[1, 0, 0]",I agree with Sherry that in a close intimate r...
2,Why do I feel like I'm always wrong in everyth...,My wife is always accusing me of cheating and ...,https://counselchat.com/questions/why-do-i-fee...,workplace-relationships,Hello. That must be very frustrating for you t...,9,268,others,"[1, 0, 1]",That must be very frustrating for you to feel ...
3,Why do I feel sad all the time?,I just feel sad all the time and I don't like ...,https://counselchat.com/questions/why-do-i-fee...,family-conflict,"Hello,While one can be sad from time to time, ...",9,264,family_conflicts,"[1, 1, 0]","If you feel sad on most days, it is worthwhile..."
4,The Underlying Causes of Being Possessive,I am extremely possessive in my relationships ...,https://counselchat.com/questions/the-underlyi...,social-relationships,Hi there. Its great you are able to realize th...,7,224,others,"[0, 0, 1]",Its great you are able to realize there are ot...


In [153]:
df["text"] = df.questionText + ' ' + df.questionTitle
df["text"] = df["text"].astype(str)
df["labels"] = df.root_multi_label

In [154]:
df.dtypes

questionTitle       object
questionText        object
questionLink        object
topic               object
answerText          object
upvotes              int64
views                int64
root_topic          object
root_multi_label    object
reflection          object
text                object
labels              object
dtype: object

In [155]:
model = MultiLabelClassificationModel('bert', 'bert-base-uncased', num_labels=3, use_cuda=False)

In [156]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state =333)

In [157]:
train_df = train_df[['text', 'labels']]
test_df = test_df[['text', 'labels']]

In [158]:
train_df = train_df.reset_index()
train_df.drop(['index'], axis = 1, inplace = True)

In [146]:
test_df = test_df.reset_index()
test_df.drop(['index'], axis = 1, inplace = True)

In [159]:
model.train_model(train_df, args={'learning_rate':1e-4, 'num_train_epochs': 10, 'reprocess_input_data': True, 'overwrite_output_dir': True,"train_batch_size": 14})

Converting to features started. Cache is not used.


Running loss: 0.347322

Running loss: 0.546854

/Users/kaushik-shakkari/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.590284

...

Running loss: 0.016176

Running loss: 0.050648

Running loss: 0.014145

Running loss: 0.007887

Running loss: 0.005918Training of bert model complete. Saved to outputs/.


In [161]:
result, model_outputs, wrong_predictions = model.eval_model(test_df)

In [162]:
result

{'LRAP': 0.9146666666666668, 'eval_loss': 0.7711860002018511}